### This file not intended for production code; all it does is calibrate the models for the LODES dataset and saves the model objects.

In [37]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import trange
from pysal.model.spint import Gravity, Production
from scipy.spatial.distance import pdist, squareform

In [9]:
# Load data (these filenames are where it is locally for me)
attrdf = pd.read_csv('../../../../data/attrs.csv', converters={'name' : str}, index_col=0)
flowdf = pd.read_csv('../../../../data/lodes-flows.csv', converters={'origin' : str, 'dest' : str}, index_col=0)

In [51]:
# Drop AK and HI (didn't do this in the collection script)
flowdf.drop(flowdf[flowdf['origin'].map(lambda x : x[:2]) == '02'].index, inplace=True)
flowdf.drop(flowdf[flowdf['dest'].map(lambda x : x[:2]) == '02'].index, inplace=True)
flowdf.drop(flowdf[flowdf['origin'].map(lambda x : x[:2]) == '15'].index, inplace=True)
flowdf.drop(flowdf[flowdf['dest'].map(lambda x : x[:2]) == '15'].index, inplace=True)

In [90]:
# Drop '46113' and '51515' which still exist for some reason
flowdf.drop(flowdf[flowdf['origin'] == '46113'].index, inplace=True)
flowdf.drop(flowdf[flowdf['dest'] == '46113'].index, inplace=True)
flowdf.drop(flowdf[flowdf['origin'] == '51515'].index, inplace=True)
flowdf.drop(flowdf[flowdf['dest'] == '51515'].index, inplace=True)

In [12]:
# Merge the dataframes
o_map = dict(attrdf[['name', 'o_attr']].values)
d_map = dict(attrdf[['name', 'd_attr']].values)
p_map = dict(attrdf[['name', 'pop']].values)

flowdf['o_attr'] = flowdf['origin'].map(o_map)
flowdf['d_attr'] = flowdf['dest'].map(d_map)
flowdf['o_pop']  = flowdf['origin'].map(p_map)
flowdf['d_pop']  = flowdf['dest'].map(p_map)

In [52]:
# Create costs via Euclidean distance
coords = attrdf[['lat', 'lon']].values
dists = pdist(coords)
cost_arr = np.zeros((flowdf.shape[0], 1))
names = attrdf['name'].values

for i in trange(flowdf.shape[0]):
    o_name = flowdf['origin'].iloc[i]
    d_name = flowdf['dest'].iloc[i]
    o_idx = np.where(names == o_name)[0][0]
    d_idx = np.where(names == d_name)[0][0]
    cost_arr[i] = dists[attrdf.shape[0] * o_idx + d_idx - ((o_idx + 2) * (o_idx + 1)) // 2]  # use formula from scipy docs

flowdf['cost'] = cost_arr    

100%|██████████| 822650/822650 [03:22<00:00, 4060.45it/s]


In [101]:
# Fit unconstrained gravity model to data
flows = flowdf['count'].values.reshape(-1, 1)
origins = flowdf[['o_attr', 'o_pop']].values
destinations = flowdf[['d_attr', 'd_pop']].values
cost = flowdf['cost'].values.reshape(-1, 1)

unconstrained = Gravity(flows, origins, destinations, cost, cost_func='pow').fit()  # better results with pow over exp

In [100]:
# Fit production-constrained model to data
production = Production(flows, flowdf['origin'].values, destinations, cost, cost_func='pow').fit()

In [102]:
unconstrained.pseudoR2

0.24023565315205087

In [103]:
production.pseudoR2

0.2459903457302859